In [198]:
import pandas as pd
import yfinance as yf
import pendulum
import csv

# Define the start and end dates for the loop
import datetime

In [199]:
# data = yf.download(
#   tickers=['AAPL'],
#   # start=start_time,
#   # end=end_time,
#   period='max',
#   interval='1d'
# )

# # Drop second level of column names
# data.columns = data.columns.droplevel(1)

# csv_file = "stock_data_alternative.csv"
# with open(csv_file, mode="w", newline="") as file:
# 		writer = csv.writer(file)
# 		# Write the header
# 		writer.writerow(["Date"] + list(data.columns))
# 		# Write the data
# 		for index, row in data.iterrows():
# 				writer.writerow([index] + list(row))


In [200]:
def clean_data(df):
    print("Initial data information:")
    print(df.info())

    # Remove unnecessary columns (if there are columns named 'Unnamed')
    df = df.loc[:, ~df.columns.get_level_values(1).str.contains('^Unnamed')]

    # Handle the 'timestamp' column (if present)
    if ('timestamp', '') in df.columns:
        # Convert to datetime format
        df[('timestamp', '')] = pd.to_datetime(df[('timestamp', '')], errors='coerce')  
        
        # Remove rows with missing timestamp values
        df = df.dropna(subset=[('timestamp', '')]) 

    # Fill missing values using forward fill and backward fill methods
    df = df.fillna(method='ffill').fillna(method='bfill')

    # Normalize 'close' values between 0 and 1 if present
    if ('close', '') in df.columns:
        df[('close', '')] = (df[('close', '')] - df[('close', '')].min()) / (df[('close', '')].max() - df[('close', '')].min())

    # Print out the data after cleaning
    print("Data after cleaning:")
    print(df.head())

    return df

data

,Adj Close,Close,High,Low,Open,Volume
Datetime,,,,,,
2024-12-07 00:00:00+00:00,99869.976562,99869.976562,99869.976562,99869.976562,99869.976562,0
2024-12-07 00:02:00+00:00,99778.046875,99778.046875,99778.046875,99778.046875,99778.046875,0
2024-12-07 00:04:00+00:00,99793.875000,99793.875000,99793.875000,99793.875000,99793.875000,0
2024-12-07 00:06:00+00:00,99911.695312,99911.695312,99911.695312,99911.695312,99911.695312,0
2024-12-07 00:07:00+00:00,100005.054688,100005.054688,100005.054688,100005.054688,100005.054688,0
...,...,...,...,...,...,...
2024-12-07 14:54:00+00:00,99546.695312,99546.695312,99546.695312,99546.695312,99546.695312,0
2024-12-07 14:55:00+00:00,99536.851562,99536.851562,99536.851562,99536.851562,99536.851562,0
2024-12-07 14:56:00+00:00,99513.968750,99513.968750,99513.968750,99513.968750,99513.968750,0


In [201]:
def fillData(df):
  # Create a new DataFrame with a complete range of dates and times
  complete_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='1min')
  complete_data = pd.DataFrame(index=complete_index)

  # Reindex the 'all_data' DataFrame with the complete index
  filledData = df.reindex(complete_data.index)

  # Reset the index to make 'Datetime' a column again
  filledData.reset_index(inplace=True)

  # Rename the 'Datetime' column to 'index'
  filledData.set_index('index', inplace=True)

  # Rename 'index' to 'Date' in the index
  filledData.index.name = 'Date'

  # replace nan values with mean of the before and after values for all columns
  filledData = filledData.fillna(all_data.mean())

  # Print the updated DataFrame
  return filledData


In [202]:
def UpdateData(filePath):
	df = pd.read_csv(filePath)
	# df = pd.read_csv('./stock_data/BTC_2010-2011.csv')

	# Convert the 'Date' column to the DataFrame index
	df['Date'] = pd.to_datetime(df['Date'])
	df.set_index('Date', inplace=True)

	# Get the end date of the data
	days_before = df.index[-1]
	days_before = days_before.date()

	# Get date time now and replace the time with 00:00:00
	current_date = datetime.date.today()

	# Create an empty DataFrame to store the data
	# all_data = pd.read_csv('stock_data/BTC_2010-2011.csv', index_col=0, header=[0, 1]).sort_index(axis=1)
	all_data = pd.DataFrame()

	# Loop through the range of dates
	for date in pd.date_range(days_before, current_date, freq='D'):
		# Define the start and end times for the data retrieval
		start_time = date.replace(hour=0, minute=0, second=0)
		end_time = date.replace(hour=23, minute=59, second=59)

		# Retrieve the data for the specified date range
		data = yf.download(
			tickers=['BTC-USD'],
			start=start_time,
			end=end_time,
			# period='1d',
			interval='1m'
		)

		# Flatten the column headers to remove the ticker
		if (isinstance(data.columns, pd.MultiIndex)):
				data.columns = [col[0] for col in data.columns]  # Retain only the first level

		# Append the data to the DataFrame
		all_data = pd.concat([all_data, data])
		
	# Update the `all_data` to include the new data
	new_data = pd.concat([filledData, all_data])

	# Remove any duplicate rows
	new_data = new_data[~new_data.index.duplicated(keep='first')]

	return fillData(new_data)

In [203]:
new_data = UpdateData('./stock_data/BTC_2010-2011.csv')

# Save the updated data to a CSV file
new_data.to_csv('./stock_data/BTC_2010-2011.csv')

[*********************100%***********************]  1 of 1 completed
